In [1]:
import model_def
import torch
import torch.nn as nn
import os
from typing import Tuple
from datetime import datetime
from torch.utils import tensorboard
import copy
import os
from utils import (
    get_dataframe,
    get_dataset,
    get_loaders,
    get_position_tensor,
    get_engineered_features,
    CustomDataset,
)
from argparse import ArgumentParser
from train import *
import pandas as pd

2024-03-21 15:33:18.387821: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-21 15:33:18.390962: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-21 15:33:18.434476: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-21 15:33:19.184617: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [47]:
test_data = pd.read_csv("../test-data/test_data_set_with_metadata.csv")
test_data.drop(columns=["totas_id"], inplace=True)
test_data

,file_name,mover_id,label,ExposureBegin,ExposureTime,CcdTemperature,BackgroundMean,BackgroundSigma,RightAscension,Declination,...,FitOrder,OffsetRightAscension,OffsetDeclination,Offset,pos_X,pos_Y,pos_Flux,pos_RightAscension,pos_Declination,pos_Magnitude
0,b0504481-1.png,TL28001,1,2021-02-09 03:00:29,30,,1513.110352,54.338047,9.653262,11.649729,...,4,3.204000,-0.516242,3.245274,280.156487,696.628927,13081,9.659367,11.578098,16.749987
1,b0504481-2.png,TL28001,1,2021-02-09 03:17:19,30,,1526.349243,54.398693,9.653512,11.649434,...,4,3.424358,-0.533969,3.465685,297.557301,692.313925,12984,9.659161,11.579464,16.747326
2,b0504481-4.png,TL28001,1,2021-02-09 03:50:59,30,,1584.739014,55.196377,9.653175,11.649028,...,4,3.127741,-0.558347,3.177135,300.392192,684.308543,12935,9.658749,11.582133,16.768602
3,b0504481-3.png,TL28001,1,2021-02-09 03:34:09,30,,1552.176514,54.760712,9.653200,11.650068,...,4,3.149436,-0.495919,3.188195,293.485312,690.492596,12629,9.658958,11.580787,16.811176
4,b0504482-1.png,OG21023,1,2021-02-09 03:00:29,30,,1513.110352,54.338047,9.653262,11.649729,...,4,3.204000,-0.516242,3.245274,34.366181,204.232653,1715,9.665846,11.768376,18.955929
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50962,b0547442-4.png,TO2C075,1,2024-02-13 01:13:19,30,,1486.889648,53.126728,11.082646,23.235836,...,4,-12.070899,15.650132,19.771620,176.210899,551.326491,4092,11.092064,23.219759,17.915482
50963,b0547448-4.png,TO2C076,1,2024-02-13 01:13:57,30,,1441.180176,58.230804,11.139033,22.840876,...,4,10.999433,12.952570,16.998471,509.478424,150.317073,533,11.139154,22.980389,20.034643
50964,b0547448-1.png,TO2C076,1,2024-02-13 00:23:29,30,,1625.897095,59.541183,11.139412,22.843153,...,4,11.313753,13.089160,17.306986,504.303944,164.914153,431,11.139676,22.977000,20.149536
50965,b0547448-2.png,TO2C076,1,2024-02-13 00:40:19,30,,1515.930176,59.007259,11.139371,22.841838,...,4,11.280169,13.010271,17.225318,509.255183,158.296651,627,11.139497,22.978217,19.928152


In [3]:
real_movers = test_data[test_data["label"] == 1]
bogus_movers = test_data[test_data["label"] == 0]

In [15]:
for mover_id, data in test_data.groupby("mover_id"):
    print(f"mover_id: {mover_id}")
    print(data)
    break

mover_id: M314606
         file_name mover_id  label
16  b0504485-3.png  M314606      0
17  b0504485-2.png  M314606      0
18  b0504485-4.png  M314606      0
19  b0504485-1.png  M314606      0


In [48]:
test_dataset, test_mover_ids = get_dataset(
    test_data.groupby("mover_id"), "../../data/centered_on_asteroids/"
)

Movers: 12741.0, Real asteroids: 4597.0, Bogus asteroids: 8144.0


# Non metadata models

In [18]:
import os

In [31]:
no_metadata_models_path = "best-models/"
best_no_metadata_model_paths = os.listdir(no_metadata_models_path)

In [41]:
def get_best_model(model_name: str, folder_path: str):
    models = os.listdir(folder_path + model_name)
    # argmax on the list from the last character of the name
    best_model = max(models, key=lambda x: int(x[-1]))
    return folder_path + model_name + "/" + best_model

In [33]:
get_best_model(best_no_metadata_model_paths[0], no_metadata_models_path)

model_20240314_220129_3


In [34]:
best_no_metadata_model_paths_filtered = [
    x.split("end-to-end-no_metadata-")[-1] for x in best_no_metadata_model_paths
]
best_no_metadata_model_paths_filtered

['ncb_2-ks_5-nf_64-fvs_20-lr_0.001-mlp_hl_2-mlp_hu_64',
 'ncb_2-ks_5-nf_16-fvs_30-lr_0.001-mlp_hl_2-mlp_hu_64',
 'ncb_2-ks_5-nf_128-fvs_10-lr_0.001-mlp_hl_2-mlp_hu_64',
 'ncb_2-ks_5-nf_64-fvs_10-lr_0.001-mlp_hl_2-mlp_hu_64',
 'ncb_2-ks_5-nf_32-fvs_10-lr_0.001-mlp_hl_2-mlp_hu_64']

In [79]:
def get_model_config(model_name: str):
    model_config_str = model_name.split("end-to-end-no_metadata-")[-1]
    model_parameters = {}
    model_parameter_list = [
        "number_convs",
        "kernel_size",
        "channels_per_conv",
        "feature_vector_size",
        "learning_rate",
        "fully_connected_layers",
        "units_per_layer",
    ]

    values_with_annotation = model_config_str.split("-")
    values = [i.split("_")[-1] for i in values_with_annotation]
    model_parameters[model_name] = dict(zip(model_parameter_list, values))
    return model_parameters

In [83]:
model_configs = {}
for i in range(len(best_no_metadata_model_paths)):
    config = get_model_config(best_no_metadata_model_paths[i])
    model_configs[best_no_metadata_model_paths[i]] = config[
        best_no_metadata_model_paths[i]
    ]
model_configs

{'end-to-end-no_metadata-ncb_2-ks_5-nf_64-fvs_20-lr_0.001-mlp_hl_2-mlp_hu_64': {'number_convs': '2',
  'kernel_size': '5',
  'channels_per_conv': '64',
  'feature_vector_size': '20',
  'learning_rate': '0.001',
  'fully_connected_layers': '2',
  'units_per_layer': '64'},
 'end-to-end-no_metadata-ncb_2-ks_5-nf_16-fvs_30-lr_0.001-mlp_hl_2-mlp_hu_64': {'number_convs': '2',
  'kernel_size': '5',
  'channels_per_conv': '16',
  'feature_vector_size': '30',
  'learning_rate': '0.001',
  'fully_connected_layers': '2',
  'units_per_layer': '64'},
 'end-to-end-no_metadata-ncb_2-ks_5-nf_128-fvs_10-lr_0.001-mlp_hl_2-mlp_hu_64': {'number_convs': '2',
  'kernel_size': '5',
  'channels_per_conv': '128',
  'feature_vector_size': '10',
  'learning_rate': '0.001',
  'fully_connected_layers': '2',
  'units_per_layer': '64'},
 'end-to-end-no_metadata-ncb_2-ks_5-nf_64-fvs_10-lr_0.001-mlp_hl_2-mlp_hu_64': {'number_convs': '2',
  'kernel_size': '5',
  'channels_per_conv': '64',
  'feature_vector_size': '10',

In [43]:
best_models = {}
for model_path in best_no_metadata_model_paths:
    best_models[model_path] = get_best_model(model_path, no_metadata_models_path)
best_models

{'end-to-end-no_metadata-ncb_2-ks_5-nf_64-fvs_20-lr_0.001-mlp_hl_2-mlp_hu_64': 'best-models/end-to-end-no_metadata-ncb_2-ks_5-nf_64-fvs_20-lr_0.001-mlp_hl_2-mlp_hu_64/model_20240314_220129_3',
 'end-to-end-no_metadata-ncb_2-ks_5-nf_16-fvs_30-lr_0.001-mlp_hl_2-mlp_hu_64': 'best-models/end-to-end-no_metadata-ncb_2-ks_5-nf_16-fvs_30-lr_0.001-mlp_hl_2-mlp_hu_64/model_20240314_214410_2',
 'end-to-end-no_metadata-ncb_2-ks_5-nf_128-fvs_10-lr_0.001-mlp_hl_2-mlp_hu_64': 'best-models/end-to-end-no_metadata-ncb_2-ks_5-nf_128-fvs_10-lr_0.001-mlp_hl_2-mlp_hu_64/model_20240314_221304_8',
 'end-to-end-no_metadata-ncb_2-ks_5-nf_64-fvs_10-lr_0.001-mlp_hl_2-mlp_hu_64': 'best-models/end-to-end-no_metadata-ncb_2-ks_5-nf_64-fvs_10-lr_0.001-mlp_hl_2-mlp_hu_64/model_20240314_215611_6',
 'end-to-end-no_metadata-ncb_2-ks_5-nf_32-fvs_10-lr_0.001-mlp_hl_2-mlp_hu_64': 'best-models/end-to-end-no_metadata-ncb_2-ks_5-nf_32-fvs_10-lr_0.001-mlp_hl_2-mlp_hu_64/model_20240314_214635_5'}

In [65]:
# Filter the test data to only include the movers from test_mover_ids
metadata_type = "no_metadata"
test_data_filtered = test_data[test_data["mover_id"].isin(test_mover_ids)]
metadata = get_position_tensor(test_data_filtered.groupby("mover_id"))
extra_features = get_engineered_features(metadata, metadata_type)
dataset = CustomDataset(
    test_dataset.tensors[0], extra_features, test_dataset.tensors[1]
)
image_shape = (30, 30)
metadata_size = len(extra_features[0])

In [87]:
def initalise_model(model_parameters: dict, model_path: str, metadata_size: int):
    model = model_def.DynamicCFN(
        image_shape=image_shape,
        num_conv_blocks=int(model_parameters["number_convs"]),
        conv_filters_list=[int(model_parameters["channels_per_conv"])]
        * int(model_parameters["number_convs"]),
        conv_kernel_size=int(model_parameters["kernel_size"]),
        feature_vector_output_size=int(model_parameters["feature_vector_size"]),
        images_per_sequence=4,
        metadata_size=metadata_size,
        hidden_mlp_layers=int(model_parameters["fully_connected_layers"]),
        hidden_mlp_size=int(model_parameters["units_per_layer"]),
    )

    print(f"Model initialised with parameters: {model_parameters}")

    model.load_state_dict(torch.load(model_path))
    return model

In [88]:
for model in best_no_metadata_model_paths:
    model_parameters = model_configs[model]
    print(model_parameters)
    model_path = best_models[model]
    model = initalise_model(model_parameters, model_path, metadata_size)
    break

{'number_convs': '2', 'kernel_size': '5', 'channels_per_conv': '64', 'feature_vector_size': '20', 'learning_rate': '0.001', 'fully_connected_layers': '2', 'units_per_layer': '64'}
Feature vector input size: 2304
Model initialised with parameters: {'number_convs': '2', 'kernel_size': '5', 'channels_per_conv': '64', 'feature_vector_size': '20', 'learning_rate': '0.001', 'fully_connected_layers': '2', 'units_per_layer': '64'}


In [91]:
def get_testset_performance(model: nn.Module, val_images: torch.Tensor, val_labels: torch.Tensor, criterion: nn.modules.loss._loss):
    

((tensor([[[0.0039, 0.0039, 0.0039,  ..., 0.0039, 0.0039, 0.0039],
           [0.0039, 0.0941, 0.0039,  ..., 0.0039, 0.0235, 0.0039],
           [0.0039, 0.0039, 0.0039,  ..., 0.0039, 0.0941, 0.0235],
           ...,
           [0.0039, 0.0784, 0.0039,  ..., 0.0471, 0.0039, 0.0667],
           [0.1294, 0.0039, 0.0392,  ..., 0.0784, 0.1608, 0.0392],
           [0.1020, 0.1216, 0.0039,  ..., 0.1490, 0.0039, 0.0157]]]),
  tensor([0])),
 tensor([0.]))